In [16]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from pathlib import Path
import numpy as np
import pandas as pd

from tensorflow import keras                            
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [17]:
import matplotlib.font_manager as fm

# 설치된 폰트 출력
font_list = [font.name for font in fm.fontManager.ttflist]
font_list
plt.rcParams['font.family'] = 'GULIM'

In [28]:
dir_ = Path('E:\PetData\Data\Training\라벨링데이터\TL2\개\안구\일반')
dir_val = Path ('E:\PetData\Data\Training\라벨링데이터\TL2\개\안구\일반')
filepaths = list(dir_.glob(r'**/*.jpg'))
filepaths_val = list (dir_val.glob(r'**/*.jpg'))
print(len(filepaths))
print(str(filepaths[6]).split("\\")[-3])
def proc_img(filepath):
    """
   		이미지데이터의 경로와 label데이터로 데이터프레임 만들기 
    """

    labels = [str(filepath[i]).split("\\")[-3] \
              for i in range(len(filepath))]
    
    filepath = pd.Series(filepath, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')
    print(labels)

    # 경로와 라벨 concatenate
    df = pd.concat([filepath, labels], axis=1)

    # index 재설정
    df = df.sample(frac=1,random_state=0).reset_index(drop = True)
    
    return df

df = proc_img(filepaths)
df_var = proc_img(filepaths_val)
df.head(5)

34748
안검내반증
0        안검내반증
1        안검내반증
2        안검내반증
3        안검내반증
4        안검내반증
         ...  
34743      핵경화
34744      핵경화
34745      핵경화
34746      핵경화
34747      핵경화
Name: Label, Length: 34748, dtype: object
0        안검내반증
1        안검내반증
2        안검내반증
3        안검내반증
4        안검내반증
         ...  
34743      핵경화
34744      핵경화
34745      핵경화
34746      핵경화
34747      핵경화
Name: Label, Length: 34748, dtype: object


,Filepath,Label
0,E:\PetData\Data\Training\라벨링데이터\TL2\개\안구\일반\안검...,안검염
1,E:\PetData\Data\Training\라벨링데이터\TL2\개\안구\일반\안검...,안검염
2,E:\PetData\Data\Training\라벨링데이터\TL2\개\안구\일반\핵경...,핵경화
3,E:\PetData\Data\Training\라벨링데이터\TL2\개\안구\일반\핵경...,핵경화
4,E:\PetData\Data\Training\라벨링데이터\TL2\개\안구\일반\핵경...,핵경화


In [29]:
#데이터 전처리
batch_size = 32
img_height = 180
img_width = 180
print(df.Label)

0          안검염
1          안검염
2          핵경화
3          핵경화
4          핵경화
         ...  
34743      유루증
34744      핵경화
34745      핵경화
34746      유루증
34747    안검내반증
Name: Label, Length: 34748, dtype: object


In [30]:
#각각 트레이닝 세트 validation set 구현
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  dir_,
  validation_split=0.8,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  dir_val,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)


Found 40466 files belonging to 5 classes.
Using 8094 files for training.
Found 40466 files belonging to 5 classes.
Using 8093 files for validation.


In [31]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [32]:
#rescaling 전처리 레이어를 생성하는 코드

# 전처리 레이어 생성, 0과 1 사이의 범위를 1/255 스케일 팩터로 사용
normalization_layer = layers.experimental.preprocessing.Rescaling(1./255) 

#이 코드는 train_ds 데이터셋의 모든 이미지에 대해 normalization_layer를 적용하여 정규화된 데이터셋 normalized_ds를 생성합니다.
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
#normalized_ds에서 첫 번째 배치의 이미지와 레이블을 가져옵니다.
image_batch, labels_batch = next(iter(normalized_ds))

first_image = image_batch[0]
# 정규화된 이미지의 최소 및 최대 픽셀 값을 출력합니다
print(np.min(first_image), np.max(first_image))

0.036491532 1.0


In [33]:
num_classes = 5
#텐서 플로우의 Sequential 모델을 사용
model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [34]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [35]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_5 (Rescaling)     (None, 180, 180, 3)       0         
                                                                 
 conv2d_21 (Conv2D)          (None, 180, 180, 16)      448       
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 90, 90, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_22 (Conv2D)          (None, 90, 90, 32)        4640      
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 45, 45, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_23 (Conv2D)          (None, 45, 45, 64)       

In [38]:
model.fit(train_ds,validation_data=val_ds,epochs=10)
#질문 사항 어째서 테스트와 validation이 같은 데이터인데 이렇게 차이가 나지?

GPU를 사용한 학습
Epoch 1/10
253/253 [==============================] - 64s 252ms/step - loss: 0.8213 - accuracy: 0.6658 - val_loss: 1.3174 - val_accuracy: 0.4897
Epoch 2/10
253/253 [==============================] - 63s 251ms/step - loss: 0.6202 - accuracy: 0.7575 - val_loss: 1.5059 - val_accuracy: 0.4925
Epoch 3/10
253/253 [==============================] - 76s 299ms/step - loss: 0.4483 - accuracy: 0.8411 - val_loss: 1.5196 - val_accuracy: 0.4757
Epoch 4/10
253/253 [==============================] - 62s 247ms/step - loss: 0.3453 - accuracy: 0.8831 - val_loss: 1.7456 - val_accuracy: 0.4757
Epoch 5/10
253/253 [==============================] - 62s 247ms/step - loss: 0.2993 - accuracy: 0.9052 - val_loss: 1.9567 - val_accuracy: 0.4594
Epoch 6/10
253/253 [==============================] - 62s 247ms/step - loss: 0.2638 - accuracy: 0.9104 - val_loss: 1.8423 - val_accuracy: 0.4821
Epoch 7/10
253/253 [==============================] - 62s 246ms/step - loss: 0.2419 - accuracy: 0.9196 - val_loss: 1.9

In [43]:
#레이어 수정 버전. Sequential 모델 CNN
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

# Set the path to your dataset
train_data_dir = 'E:\PetData\Data\Training\라벨링데이터\TL2\개\안구\일반'
test_data_dir = 'E:\PetData\Data\Training\라벨링데이터\TL2\개\안구\일반'

# Set the image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32

# Data augmentation for training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# Rescaling for test set
test_datagen = ImageDataGenerator(rescale=1./255)

# Load and preprocess the training data
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

# Load and preprocess the test data
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

# Create the model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(train_generator.num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

# Train the model
model.fit(train_generator,
          steps_per_epoch=train_generator.samples // batch_size,
          epochs=10,
          validation_data=test_generator,
          validation_steps=test_generator.samples // batch_size)

# Evaluate the model
score = model.evaluate(test_generator)
print(f'Test loss: {score[0]}')
print(f'Test accuracy: {score[1]}')

Found 40466 images belonging to 5 classes.
Found 40466 images belonging to 5 classes.
Epoch 1/10
1264/1264 [==============================] - 763s 603ms/step - loss: 1.2068 - accuracy: 0.4728 - val_loss: 1.1371 - val_accuracy: 0.5041
Epoch 2/10
1264/1264 [==============================] - 452s 357ms/step - loss: 1.1155 - accuracy: 0.5148 - val_loss: 1.0768 - val_accuracy: 0.5311
Epoch 3/10
1264/1264 [==============================] - 457s 362ms/step - loss: 1.0735 - accuracy: 0.5333 - val_loss: 1.0199 - val_accuracy: 0.5610
Epoch 4/10
1264/1264 [==============================] - 448s 354ms/step - loss: 1.0458 - accuracy: 0.5481 - val_loss: 1.0193 - val_accuracy: 0.5625
Epoch 5/10
1264/1264 [==============================] - 440s 348ms/step - loss: 1.0189 - accuracy: 0.5633 - val_loss: 0.9714 - val_accuracy: 0.5822
Epoch 6/10
1264/1264 [==============================] - 445s 352ms/step - loss: 0.9934 - accuracy: 0.5719 - val_loss: 0.9405 - val_accuracy: 0.5985
Epoch 7/10
1264/1264 [====

In [44]:
# CNN ResNet 버전
import tensorflow as tf
from tensorflow.keras.applications.resnet_v2 import ResNet152V2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 데이터 경로 설정
train_data_dir = 'E:\PetData\Data\Training\라벨링데이터\TL2\개\안구\일반'
valid_data_dir = 'E:\PetData\Data\Training\라벨링데이터\TL2\개\안구\일반'
test_data_dir = 'E:\PetData\Data\Validation\LabelingData\VL\Dog\Eye\common'

# 이미지 사이즈 및 배치 사이즈 설정
img_height, img_width = 224, 224
batch_size = 32

# 데이터 증강을 위한 ImageDataGenerator 설정
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# 훈련 데이터 로드 및 데이터 증강 적용
train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(img_height, img_width),
                                                    batch_size=batch_size,
                                                    class_mode='categorical')
                                                    
# 검증 데이터 로드 (증강 적용하지 않음)
valid_generator = valid_datagen.flow_from_directory(valid_data_dir,
                                                    target_size=(img_height, img_width),
                                                    batch_size=batch_size,
                                                    class_mode='categorical')

# 테스트 데이터 로드 (증강 적용하지 않음)
test_generator = test_datagen.flow_from_directory(test_data_dir,
                                                  target_size=(img_height, img_width),
                                                  batch_size=batch_size,
                                                  class_mode='categorical')

# 클래스 개수 추출
num_classes = train_generator.num_classes

# ResNet_v2_152 모델 불러오기 (pre-trained weights 사용)
base_model = ResNet152V2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# 모델의 마지막 레이어 수정
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# 전체 모델 정의
model = Model(inputs=base_model.input, outputs=predictions)

# 기존의 가중치를 동결
for layer in base_model.layers:
    layer.trainable = False

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(train_generator,
          steps_per_epoch=train_generator.samples // batch_size,
          validation_data=valid_generator,
          validation_steps=valid_generator.samples // batch_size,
          epochs=10)

# 모델 평가
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

Found 40466 images belonging to 5 classes.
Found 40466 images belonging to 5 classes.
Found 6671 images belonging to 5 classes.
Epoch 1/10
1264/1264 [==============================] - 6321s 5s/step - loss: 1.1493 - accuracy: 0.5196 - val_loss: 1.0357 - val_accuracy: 0.5603
Epoch 2/10
1264/1264 [==============================] - 6284s 5s/step - loss: 1.0361 - accuracy: 0.5622 - val_loss: 0.9802 - val_accuracy: 0.5853
Epoch 3/10
1264/1264 [==============================] - 6264s 5s/step - loss: 0.9963 - accuracy: 0.5769 - val_loss: 0.9660 - val_accuracy: 0.5931
Epoch 4/10
   8/1264 [..............................] - ETA: 48:39 - loss: 0.8646 - accuracy: 0.6445

KeyboardInterrupt: 